In [14]:
# https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=hQ1oK0kXaV5p
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import numpy as np
import random
from torch.utils.data import Dataset, random_split

# Load pretrained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("antoiloui/belgpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("antoiloui/belgpt2", model_max_length=768, pad_token='<|pad|>')


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [2]:
replace = [ 
    ( '\t', ' ')
]

# until line 1645 there are dialogues
# lets split it there
# then creating pairs or triplets or 4lets or whatever -> check length
# for the model
# using the rest of the text just as a normal finetune training set

for c in ['A', 'B', 'C']:
    # get rid of weird whitespace
    replace.append(('{} : '.format(c), '{}: '.format(c)))
    # adding missing newlines
    replace.append((' {}: '.format(c), '\n{}: '.format(c)))
    
print(replace)

# load dataset and split into smaller texts
with open('datasets/training-raw.txt', 'r') as raw_file:
    text = raw_file.read()
    for t,r in replace:
        text = text.replace(t, r)
split_text = text.split('\n\n')
# dialogs end after 1678
split_after_dialog = text.split('\n')[:1678]
just_text = list(filter(None, text.split('\n')[1678:]))

# [::2] samples every second item, [1::2] starts at index one

pairs = map( lambda x: '\n'.join(x), zip(split_after_dialog[::2], split_after_dialog[1::2]))
pairs = list(pairs)

print(pairs[1])
print(len(text))
print(len(split_after_dialog))


[('\t', ' '), ('A : ', 'A: '), (' A: ', '\nA: '), ('B : ', 'B: '), (' B: ', '\nB: '), ('C : ', 'C: '), (' C: ', '\nC: ')]
B: Dieu? Écoutez, je ne veux pas vous vexer, mais je ne crois pas que Dieu existe.
A: Vous ne croyez pas que Dieu existe? Comment cela?
5090210
1678


In [3]:
print(tokenizer.pad_token_id)
print(tokenizer.eos_token_id)
train_encodings = tokenizer(text, truncation=True)
print(train_encodings)


50258
50257
{'input_ids': [174, 122, 126, 8216, 1702, 559, 1184, 295, 30959, 337, 338, 1603, 303, 24654, 16, 5079, 15, 352, 267, 161, 225, 250, 4029, 2622, 297, 37017, 1357, 354, 287, 31215, 1710, 338, 6392, 16, 980, 866, 287, 1603, 303, 24654, 11805, 14, 287, 1702, 385, 6177, 364, 287, 7281, 16, 41634, 39947, 14, 581, 2884, 451, 1589, 15, 36957, 293, 9499, 295, 30959, 16, 201, 35, 28, 4516, 1159, 14, 585, 1346, 16, 859, 354, 2394, 472, 13941, 4583, 3, 201, 36, 28, 2394, 33, 41276, 14, 475, 423, 3496, 380, 472, 34217, 274, 14, 489, 475, 423, 2762, 380, 354, 2394, 2723, 16, 201, 35, 28, 1146, 423, 15113, 380, 354, 2394, 2723, 33, 2918, 1061, 33, 201, 36, 28, 13557, 14, 268, 161, 225, 250, 319, 2065, 16, 897, 299, 161, 225, 250, 67, 289, 161, 225, 250, 616, 3429, 364, 287, 2386, 337, 1205, 354, 2394, 299, 161, 225, 250, 4029, 828, 380, 16, 26075, 15, 6125, 14, 506, 2394, 2723, 14, 2337, 524, 266, 15, 86, 15, 296, 1499, 311, 1773, 9313, 33, 5301, 743, 622, 1313, 23261, 33, 5301, 307, 1001

In [4]:
doc_lengths = []

for pair in pairs:
    # get rough token count distribution
    tokens = tokenizer(pair)

    doc_lengths.append(len(tokens['input_ids']))

doc_lengths = np.array(doc_lengths)
print(np.average(doc_lengths))
print(np.max(doc_lengths))
print(np.min(doc_lengths))

# the dialogs are okay because they are shorter
# then the max length

print("----")

text_lengths = []

for t in just_text:
    # get rough token count distribution
    tokens = tokenizer(t)
    if len(tokens['input_ids']) == 0:
        print(t)

    text_lengths.append(len(tokens['input_ids']))

doc_lengths = np.array(text_lengths)
print(np.average(text_lengths))
print(np.max(text_lengths))
print(np.min(text_lengths))

# the normal texts are too long
# we need to split it

Token indices sequence length is longer than the specified maximum sequence length for this model (877 > 768). Running this sequence through the model will result in indexing errors


61.94874851013111
613
1
----
1114.2517605633802
941333
2


In [5]:
max_length = 768
split_french = []

for f in just_text:
    split_french.append( [j for j in (f[i:i + max_length] for i in  range(0, len(just_text), max_length))])

# split_french = filter(None, split_french)
split_french = np.array(split_french).flatten()
split_french = [ x for x in split_french if x != '']

text_lengths = []

for t in split_french:
    # get rough token count distribution
    tokens = tokenizer(t)
    if len(tokens['input_ids']) == 0:
        print("empty {}".format(t))
    if t == '':
        print("empty string {}".format(t))

    text_lengths.append(len(tokens['input_ids']))

print(len(split_french))
print(len(split_french) > len(just_text))
doc_lengths = np.array(text_lengths)
print(np.average(text_lengths))
print(np.max(text_lengths))
print(np.min(text_lengths))

1251
True
69.98880895283773
252
2


In [15]:
batch_size = 2

class FrenchDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        
        for text in texts:
            encodings_dict = tokenizer(tokenizer.bos_token + text + tokenizer.eos_token, truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __getitem__(self, idx):
        #  {"input_ids": [...], "labels": [...], "attention_mask": [..]}
        return { "input_ids": self.input_ids[idx], "labels": self.input_ids[idx], "attention_mask": self.attn_masks[idx] }

    def __len__(self):
        return len(self.input_ids)
    


In [16]:
french_dataset = FrenchDataset(split_french, tokenizer)

# Split into training and validation sets
train_size = int(0.9 * len(french_dataset))
val_size = len(french_dataset) - train_size

train_dataset, val_dataset = random_split(french_dataset, [train_size, val_size])
# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,12.176900
20,10.374500
30,8.361200
40,5.655300
50,1.843400
60,0.659900
70,0.457300
80,0.610400
90,0.438200
100,0.319200


TrainOutput(global_step=563, training_loss=0.9867052759306892, metrics={'train_runtime': 628.6061, 'train_samples_per_second': 0.896, 'total_flos': 645103927296000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': -154378240, 'init_mem_gpu_alloc_delta': 510354432, 'init_mem_cpu_peaked_delta': 154378240, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 11145216, 'train_mem_gpu_alloc_delta': 1493839360, 'train_mem_cpu_peaked_delta': 462319616, 'train_mem_gpu_peaked_delta': 5273303040})

In [12]:
del train_dataset
del trainer
del training_args
del model
del tokenizer
del val_dataset
torch.cuda.empty_cache()